# Actividad de práctica Brain Tumor Dataset

In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from skopt import BayesSearchCV
from skopt.space import Real
from sklearn.metrics import roc_auc_score, classification_report

In [2]:
data = pd.read_csv("brain_tumor_dataset.csv")
data

,Patient_ID,Age,Gender,Tumor_Type,Tumor_Size,Location,Histology,Stage,Symptom_1,Symptom_2,Symptom_3,Radiation_Treatment,Surgery_Performed,Chemotherapy,Survival_Rate,Tumor_Growth_Rate,Family_History,MRI_Result,Follow_Up_Required
0,1,73,Male,Malignant,5.375612,Temporal,Astrocytoma,III,Vision Issues,Seizures,Seizures,No,No,No,51.312579,0.111876,No,Positive,Yes
1,2,26,Male,Benign,4.847098,Parietal,Glioblastoma,II,Headache,Headache,Nausea,Yes,Yes,Yes,46.373273,2.165736,Yes,Positive,Yes
2,3,31,Male,Benign,5.588391,Parietal,Meningioma,I,Vision Issues,Headache,Seizures,No,No,No,47.072221,1.884228,No,Negative,No
3,4,29,Male,Malignant,1.436600,Temporal,Medulloblastoma,IV,Vision Issues,Seizures,Headache,Yes,No,Yes,51.853634,1.283342,Yes,Negative,No
4,5,54,Female,Benign,2.417506,Parietal,Glioblastoma,I,Headache,Headache,Seizures,No,No,Yes,54.708987,2.069477,No,Positive,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,19996,21,Male,Malignant,9.612013,Parietal,Medulloblastoma,III,Seizures,Nausea,Vision Issues,No,No,Yes,58.229662,0.353806,No,Negative,Yes
19996,19997,32,Female,Benign,1.543560,Temporal,Meningioma,III,Headache,Headache,Vision Issues,Yes,Yes,No,77.706856,2.341074,No,Positive,No
19997,19998,57,Female,Benign,3.618634,Temporal,Medulloblastoma,I,Seizures,Vision Issues,Nausea,No,No,Yes,89.543803,2.332881,No,Positive,Yes
19998,19999,68,Male,Malignant,8.519086,Parietal,Glioblastoma,III,Seizures,Headache,Vision Issues,Yes,Yes,Yes,83.306781,2.387202,No,Positive,No


In [3]:
X = data.drop(columns=["Tumor_Type", "Patient_ID"])  
y = data["Tumor_Type"]

In [4]:
num_features = ["Age", "Tumor_Size", "Survival_Rate", "Tumor_Growth_Rate"]
cat_features = ["Gender","Location","Histology","Stage","Symptom_1","Symptom_2","Symptom_3","Radiation_Treatment","Surgery_Performed","Chemotherapy",
"Family_History","MRI_Result","Follow_Up_Required"]

In [5]:
preprocessor = ColumnTransformer([
    ("num", StandardScaler(), num_features),
    ("cat", OneHotEncoder(handle_unknown="ignore"), cat_features)
])

In [6]:
svc_pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("svc", SVC(kernel="linear", C=1))
])

In [7]:
param_space = {
    "svc__C": Real(0.1, 1, prior="log-uniform")
}

bayes_search = BayesSearchCV(
    svc_pipeline,
    search_spaces=param_space,
    n_iter=30,
    cv=10,
    scoring="roc_auc",
    n_jobs=-1,
    verbose=1,
    random_state=42
)

In [10]:
bayes_search.fit(X[:101],y[:101])

Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for eac

BayesSearchCV(cv=10,
              estimator=Pipeline(steps=[('preprocessor',
                                         ColumnTransformer(transformers=[('num',
                                                                          StandardScaler(),
                                                                          ['Age',
                                                                           'Tumor_Size',
                                                                           'Survival_Rate',
                                                                           'Tumor_Growth_Rate']),
                                                                         ('cat',
                                                                          OneHotEncoder(handle_unknown='ignore'),
                                                                          ['Gender',
                                                                           'Location',
                                                                           'Histology',
                                                                           'Stage',
                                                                           'Symptom_1',
                                                                           'Symptom_2',
                                                                           'Symptom_3',
                                                                           'Radiation_Treatment',
                                                                           'Surgery_Performed',
                                                                           'Chemotherapy',
                                                                           'Family_History',
                                                                           'MRI_Result',
                                                                           'Follow_Up_Required'])])),
                                        ('svc', SVC(C=1, kernel='linear'))]),
              n_iter=30, n_jobs=-1, random_state=42, scoring='roc_auc',
              search_spaces={'svc__C': Real(low=0.1, high=1, prior='log-uniform', transform='normalize')},
              verbose=1)

In [22]:
print("Mejor parámetro C encontrado:", bayes_search.best_params_)
print("Mejor AUC promedio:", bayes_search.best_score_)

Mejor parámetro C encontrado: OrderedDict({'svc__C': 0.10083941261235432})
Mejor AUC promedio: 0.4916666666666666
